# Part 4: Becoming a Backprop Ninja

Viele ML Frameworks bieten direkt "AutoGrad" an. Z.B. PyTorch, TensorFlow, MXNet, etc.

Wir wollen nun aber selbst eine Implementierung vornehmen, um ein echter Backpropagation Ninja zu werden.

Als Beispiel verwenden wir wieder ein MLP das Trigramme der Vornamen bekommt und den 4. Buchstaben vorhersagen soll.

In [17]:
import torch
import torch.nn.functional as F # torch.nn.functional is a module that contains all the functions in the torch.nn library
import matplotlib.pyplot as plt
import random

### Datenvorbereitung

In [27]:
words = open("names.txt", "r").read().splitlines()
print(len(words))
print(words[:10])

32033
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']


In [6]:
chars = sorted(set(''.join(words)))
print(chars)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [8]:
char2idx = {c: i+1 for i, c in enumerate(chars)}
char2idx['.'] = 0
print(char2idx)

idx2char = {i: c for c, i in char2idx.items()}
print(idx2char)

vocab_size = len(idx2char)
print(vocab_size)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [35]:
block_size = 3

def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + ".":
            ix = char2idx[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

random.seed(42)
shuffled_words = words.copy()
random.shuffle(shuffled_words)
n1 = int(0.8 * len(shuffled_words))
n2 = int(0.9 * len(shuffled_words))

Xtr, Ytr = build_dataset(words[:n1]) # 80% training
print("Train Length: ", len(Xtr))
Xdev, Ydev = build_dataset(words[n1:n2]) # 10% development
print("Dev   Length: ", len(Xdev))
Xte, Yte = build_dataset(words[n2:]) # 10% test
print("Test  Length: ", len(Xte))

Train Length:  182437
Dev   Length:  22781
Test  Length:  22928


### Aufbau des NNs

In [38]:
# Utility Function um Gradienten manuell zu berchnen und mit den AutoGrad's von PyTroch zu vergleichen
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item() # 1. Variante checkt ob die beiden Tensoren gleich sind
    app = torch.allclose(dt, t.grad)    # 2. Variante checkt ob die beiden Tensoren fast gleich sind
    maxdiff = (dt - t.grad).abs().max().item() # gibt den maximalen Unterschied zwischen den beiden Tensoren zurück
    print(f"{s:15s} | exact: {str(ex):5s} | approx: {str(app):5s} | maxdiff: {maxdiff}")

In [101]:
n_embd = 10
n_hidden = 64

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)

# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3) / ((n_embd * block_size) ** 0.5) # fan_in = n_embd * block_size, damit Gewichte normalverteilt sind, wegen Sättigung der tanh
b1 = torch.randn((n_hidden,), generator=g) * 0.1 # eig wayne, weil durch BatchNorm Bias ersetzt
# Layer 2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn((vocab_size,), generator=g) * 0.1
# BatchNorm
bngain = torch.randn((1, n_hidden), generator=g) * 0.1 + 1.0 # auch als "gamma" bezeichnet, skaliert die Normalverteilung der Aktivierung, damit sie nicht zu klein wird
bnbias = torch.randn((1, n_hidden), generator=g) * 0.1 # auch als "beta" bezeichnet, verschiebt die Normalverteilung der Aktivierung

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

4137


In [102]:
# Erstellen eines Mini-Batches
batch_size = 32
n = batch_size
# Parameter stehen für: low, high, size, generator -> random index im Bereich von 0 bis Xtr.shape[0], davon 32 Stück
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [105]:
# Forward Pass
 
emb = C[Xb] # (32, 3, 10)
embcat = emb.view(emb.shape[0], -1) # (32, 30)

# Linear Layer 1
zprebn = embcat @ W1 + b1 # (32, 64) -> Hidden Layer Pre-Activation z

# BatchNorm Layer (Eingaben vom Ersten Hidden Layer werden normalisiert, damit die Aktivierungsfunktion nicht zu klein oder zu groß wird / Gradienten nicht zu klein werden, wegen der Sättigung der tanh)
bnmeani = 1/n * zprebn.sum(dim=0, keepdim=True) # (1, 64) # von allen 32 Samples den Mittelwert berechnen
bndiff = zprebn - bnmeani # (32, 64) # von allen 32 Samples den Mittelwert abziehen -> Abweichung
bndiff2 = bndiff ** 2 # (32, 64) # Abweichung quadrieren -> Varianz
bnvar = 1/(n-1) * bndiff2.sum(dim=0, keepdim=True) # (1, 64) # Varianz berechnen (n-1 ist Bessels Korrektur, weil wir sonst die Varianz tendenziell unterschätzen)
bnvar_inv = (bnvar + 1e-5) ** -0.5 # (1, 64) # Inverse Wurzel der Varianz berechnen, für Normalisierung
bnraw = bndiff * bnvar_inv # (32, 64)
apreact = bngain * bnraw + bnbias # (32, 64) # Skalierung und Verschiebung der Normalisierten Werte, um die Aktivierung zu normalisieren
# Nicht-Lineare Aktivierungsfunktion
a = torch.tanh(apreact) # (32, 64) # tanh Aktivierungsfunktion

# Linear Layer 2
logits = a @ W2 + b2 # (32, 64) @ (64, 27) + (27,) -> (32, 27)

# Cross Entropy Loss (Softmax + Negative Log Likelihood)
logit_maxes = logits.max(dim=1, keepdim=True).values # (32, 1) # Maximalen Wert von jedem Sample berechnen
norm_logits = logits - logit_maxes # (32, 27) # Logits normalisieren, damit es keine Overflows gibt, weil das kommt danach ja in Softmax, also exp, und sonst würden wir inf bekommen
counts = norm_logits.exp() # (32, 27) # Exponentialfunktion auf die normalisierten Logits anwenden
counts_sum = counts.sum(dim=1, keepdim=True) # (32, 1) # Summe der Exponentialfunktionen berechnen
counts_sum_inv = counts_sum ** -1 # (32, 1) # Inverse der Summe berechnen
probs = counts * counts_sum_inv # (32, 27) # Wahrscheinlichkeiten berechnen
logprobs = probs.log() # (32, 27) # Logarithmus der Wahrscheinlichkeiten berechnen
loss = -logprobs[range(n), Yb].mean() # (32,) # Negative Log Likelihood berechnen


# Pytorch Autograd
for p in parameters:
    p.grad = None

for t in [logprobs, probs, counts_sum_inv, counts_sum, counts, norm_logits, logit_maxes, logits, a, apreact, bnraw, bnvar_inv, bndiff2, bndiff, bnmeani, zprebn, embcat, emb]:
    t.retain_grad() # PyTorch soll die Gradienten der Tensoren speichern

loss.backward()
loss

tensor(3.4020, grad_fn=<NegBackward0>)

Nun gehen wir angfangen bei `loss` alle Teile der Cost-Function durch und bilden die Ableitungen von Hand. Wir machen Backpropagation vom Scratch:

In [115]:
# loss = -logprobs[range(n), Yb].mean()
# hier sind die 32 Outputs drin aus Softmax bei den Samples und dann quasi durch 32 geteilt und alle negativ, weil wir ja die negative Log Likelihood berechnen
# loss = -(a + b + c) / 3    <-- 3 hier weil wir durch die Summe teilen für mean()
# dloss/da = -1/3a + -1/3b + -1/3c 
# dlass/da = -1/3
# dloss/db = -1/3
# dloss/dc = -1/3
# dloss/dx = -1/x*n
# Das gilt natürlich nun für alle 32 Werte, wobei es ja eig (32, 27) waren, die werden aber nicht verwendet wegen unserem Yb mapping, also alle "0", weil quasi Konstanten
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0 / n
dlogprobs

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000, -0.0312,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000],
        [-0.0312,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000, -0.0312,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000

In [117]:
# überprüfen mit Utiltiy Function cmp()
cmp("logprobs", dlogprobs, logprobs)

logprobs        | exact: True  | approx: True  | maxdiff: 0.0


Unser eigener Gradient für `logprobs`, passt genau zu dem vom PyTorch AutoGrad.